In [1]:
import os
import uuid
from delta import *
from delta.tables import DeltaTable
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, max, current_timestamp, lit, when, expr, udf, coalesce, current_date

In [2]:
# config 
path_suffix  = 20
delta_path = f"{os.getcwd()}/data/delta/test{path_suffix}"

In [3]:

builder = SparkSession.builder.appName("SilverLayerCheckTest2") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

24/02/12 20:45:37 WARN Utils: Your hostname, saithihas-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.100.25 instead (on interface en0)
24/02/12 20:45:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/saithihazaw/.ivy2/cache
The jars for the packages stored in: /Users/saithihazaw/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-34449b82-f421-4861-93b2-7d8650af3528;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central


:: loading settings :: url = jar:file:/opt/homebrew/Caskroom/miniforge/base/envs/delta_env/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


:: resolution report :: resolve 92ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-34449b82-f421-4861-93b2-7d8650af3528
	confs: [default]
	0 artifacts copied, 3 already retrieved (0kB/3ms)
24/02/12 20:45:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log l

In [4]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [5]:
# Sample data (replace with your actual data)
bronze_user_data = [
    ("user1", "2024-02-05","2024-02-05", "inactive", "2024-02-05"),
    ("user3", "2024-02-04","2024-02-04", "new", None),
    ("user4", "2024-02-01","2024-02-05", "inactive", "2024-02-05"),
    ("user5", "2023-02-05","2024-02-05", "active", None),
]
silver_user_data = [
    ("user1", "2023-02-05","2024-02-02", "active", None),
    ("user2", "2023-02-02","2024-02-02", "pending", None),
    ("user3", "2022-02-04","2023-02-04", "inactive", "2024-02-04"),
]

In [6]:
# Create DataFrames [seed data]
bronze_user_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("created_at", StringType(), True),
    StructField("updated_at", StringType(), True),
    StructField("user_status", StringType(), True),
    StructField("deleted_at", StringType(), True),
])
bronze_user = spark.createDataFrame(bronze_user_data, bronze_user_schema)

In [7]:
# [prestored bronze layer data] 
silver_user_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("created_at", StringType(), True),
    StructField("updated_at", StringType(), True),
    StructField("user_status", StringType(), True),
    StructField("deleted_at", StringType(), True),
])
silver_user = spark.createDataFrame(silver_user_data, silver_user_schema)

In [8]:
def perpare_scd_type2(df): 
    df = df.withColumn("id", expr("uuid()"))
    df = df.withColumn("current_flag", when(col("deleted_at").isNull(), lit("Y")).otherwise(lit("N")))
    df = df.withColumn("effective_date", current_timestamp())
    df = df.withColumn("expired_date", lit(None).cast("timestamp"))
    df = df.withColumn("expired_date", when(col("deleted_at").isNull(), lit(None).cast("timestamp")).otherwise(current_timestamp()))
    return df

In [9]:
if not os.path.exists(f"{delta_path}/bronze_user"): 
    bronze_users = perpare_scd_type2(bronze_user)
else: 
    bronze_user_data2 = [
    ("user6", "2024-02-10","2024-02-10", "new", None),
    ("user7", "2024-02-10","2024-02-10", "new", None),
    ]
    bronze_users = spark.createDataFrame(bronze_user_data2, bronze_user_schema)
    bronze_users = bronze_users.filter(col("created_at") == '2024-02-11')
    if bronze_users.count() == 0:
        bronze_users = bronze_users.withColumn("id", lit(None))
        bronze_users = bronze_users.withColumn("current_flag", lit(None))
        bronze_users = bronze_users.withColumn("effective_date", lit(None))
        bronze_users = bronze_users.withColumn("expired_date", lit(None))

In [10]:
bronze_users.show(10,False,False)

+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
|user_id|created_at|updated_at|user_status|deleted_at|id |current_flag|effective_date|expired_date|
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+



In [11]:
if DeltaTable.isDeltaTable(spark, f"{delta_path}/silver_user"):
    silverDeltaTable = DeltaTable.forPath(spark, f"{delta_path}/silver_user")
    silver_users = silverDeltaTable.toDF()
else:
    silver_users = perpare_scd_type2(silver_user)

In [12]:
silver_users.show(10,False,False)

24/02/12 20:45:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+----------+----------+-----------+----------+------------------------------------+------------+--------------------------+--------------------------+
|user_id|created_at|updated_at|user_status|deleted_at|id                                  |current_flag|effective_date            |expired_date              |
+-------+----------+----------+-----------+----------+------------------------------------+------------+--------------------------+--------------------------+
|user4  |2024-02-01|2024-02-05|inactive   |2024-02-05|ec52cf68-1881-476a-b68b-96d1d85c5a65|N           |2024-02-11 13:29:16.612573|2024-02-11 13:29:16.612573|
|user5  |2023-02-05|2024-02-05|active     |NULL      |fedf0115-ff10-4f5e-bd9b-32976415cd56|Y           |2024-02-11 13:29:16.612573|NULL                      |
|user3  |2022-02-04|2023-02-04|inactive   |2024-02-04|1b0232de-8177-44f2-8f51-708c359f4a90|Y           |2024-02-11 13:29:12.720104|NULL                      |
|user1  |2023-02-05|2024-02-02|active     |NUL

In [13]:
bronze_users.createOrReplaceTempView("bronze_user")
silver_users.createOrReplaceTempView("silver_user")

In [14]:
spark.sql("""
    select * from bronze_user
    """).show()

+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
|user_id|created_at|updated_at|user_status|deleted_at| id|current_flag|effective_date|expired_date|
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+



# New Income data 
- check with left anti join
- add new uuid
- change flag 

In [15]:
new_records = bronze_users.join(silver_users, "user_id", "left_anti")

In [16]:
new_records.show()

+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
|user_id|created_at|updated_at|user_status|deleted_at| id|current_flag|effective_date|expired_date|
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+



In [17]:
if new_records.count() != 0: 
    new_records = new_records\
                    .withColumn("expired_date", when(col("deleted_at")\
                    .isNotNull(), current_timestamp())\
                    .otherwise(col("expired_date")))
else: 
    new_records = new_records

In [18]:
new_records.show(10,False,False)

+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
|user_id|created_at|updated_at|user_status|deleted_at|id |current_flag|effective_date|expired_date|
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+



In [19]:
bronze_users.show(10,False,False)

+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
|user_id|created_at|updated_at|user_status|deleted_at|id |current_flag|effective_date|expired_date|
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+



In [20]:
silver_users.show(10,False,False)

+-------+----------+----------+-----------+----------+------------------------------------+------------+--------------------------+--------------------------+
|user_id|created_at|updated_at|user_status|deleted_at|id                                  |current_flag|effective_date            |expired_date              |
+-------+----------+----------+-----------+----------+------------------------------------+------------+--------------------------+--------------------------+
|user4  |2024-02-01|2024-02-05|inactive   |2024-02-05|ec52cf68-1881-476a-b68b-96d1d85c5a65|N           |2024-02-11 13:29:16.612573|2024-02-11 13:29:16.612573|
|user5  |2023-02-05|2024-02-05|active     |NULL      |fedf0115-ff10-4f5e-bd9b-32976415cd56|Y           |2024-02-11 13:29:16.612573|NULL                      |
|user3  |2022-02-04|2023-02-04|inactive   |2024-02-04|1b0232de-8177-44f2-8f51-708c359f4a90|Y           |2024-02-11 13:29:12.720104|NULL                      |
|user1  |2023-02-05|2024-02-02|active     |NUL

# Check update data here 
- check with user_id
- check with updated_at

In [21]:
new_updated_data = spark.sql(
    """
    select 
    bu.*
    from 
    bronze_user bu
    join 
    silver_user su on bu.user_id = su.user_id 
    where bu.updated_at <> su.updated_at
    """
)

In [22]:
new_updated_data.show(10,False,False)

+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
|user_id|created_at|updated_at|user_status|deleted_at|id |current_flag|effective_date|expired_date|
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+



In [23]:
silver_users.show(10,False,False)

+-------+----------+----------+-----------+----------+------------------------------------+------------+--------------------------+--------------------------+
|user_id|created_at|updated_at|user_status|deleted_at|id                                  |current_flag|effective_date            |expired_date              |
+-------+----------+----------+-----------+----------+------------------------------------+------------+--------------------------+--------------------------+
|user4  |2024-02-01|2024-02-05|inactive   |2024-02-05|ec52cf68-1881-476a-b68b-96d1d85c5a65|N           |2024-02-11 13:29:16.612573|2024-02-11 13:29:16.612573|
|user5  |2023-02-05|2024-02-05|active     |NULL      |fedf0115-ff10-4f5e-bd9b-32976415cd56|Y           |2024-02-11 13:29:16.612573|NULL                      |
|user3  |2022-02-04|2023-02-04|inactive   |2024-02-04|1b0232de-8177-44f2-8f51-708c359f4a90|Y           |2024-02-11 13:29:12.720104|NULL                      |
|user1  |2023-02-05|2024-02-02|active     |NUL

In [24]:
bronze_users.show(10,False,False)

+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
|user_id|created_at|updated_at|user_status|deleted_at|id |current_flag|effective_date|expired_date|
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+



In [25]:
# to get the updated record 
update_new_record = new_records.union(new_updated_data)

In [26]:
update_new_record.show(10,False,False)

+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
|user_id|created_at|updated_at|user_status|deleted_at|id |current_flag|effective_date|expired_date|
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+



In [27]:
bronze_users.show(10,False,False)

+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
|user_id|created_at|updated_at|user_status|deleted_at|id |current_flag|effective_date|expired_date|
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+



In [28]:
silver_users.show(10,False,False)

+-------+----------+----------+-----------+----------+------------------------------------+------------+--------------------------+--------------------------+
|user_id|created_at|updated_at|user_status|deleted_at|id                                  |current_flag|effective_date            |expired_date              |
+-------+----------+----------+-----------+----------+------------------------------------+------------+--------------------------+--------------------------+
|user4  |2024-02-01|2024-02-05|inactive   |2024-02-05|ec52cf68-1881-476a-b68b-96d1d85c5a65|N           |2024-02-11 13:29:16.612573|2024-02-11 13:29:16.612573|
|user5  |2023-02-05|2024-02-05|active     |NULL      |fedf0115-ff10-4f5e-bd9b-32976415cd56|Y           |2024-02-11 13:29:16.612573|NULL                      |
|user3  |2022-02-04|2023-02-04|inactive   |2024-02-04|1b0232de-8177-44f2-8f51-708c359f4a90|Y           |2024-02-11 13:29:12.720104|NULL                      |
|user1  |2023-02-05|2024-02-02|active     |NUL

In [29]:
if not os.path.exists(f"{delta_path}/bronze_user"):
    bronze_users.write.mode(saveMode="overwrite").format("delta").save(f"data/delta/test{path_suffix}/bronze_user")
if not os.path.exists(f"{delta_path}/silver_user"):
    silver_users.write.mode(saveMode="overwrite").format("delta").save(f"data/delta/test{path_suffix}/silver_user")

In [30]:
silverDeltaTable = DeltaTable.forPath(spark, f"data/delta/test{path_suffix}/silver_user")

In [31]:
silverDeltaTable.toDF().show(10,False,False)

+-------+----------+----------+-----------+----------+------------------------------------+------------+--------------------------+--------------------------+
|user_id|created_at|updated_at|user_status|deleted_at|id                                  |current_flag|effective_date            |expired_date              |
+-------+----------+----------+-----------+----------+------------------------------------+------------+--------------------------+--------------------------+
|user4  |2024-02-01|2024-02-05|inactive   |2024-02-05|ec52cf68-1881-476a-b68b-96d1d85c5a65|N           |2024-02-11 13:29:16.612573|2024-02-11 13:29:16.612573|
|user5  |2023-02-05|2024-02-05|active     |NULL      |fedf0115-ff10-4f5e-bd9b-32976415cd56|Y           |2024-02-11 13:29:16.612573|NULL                      |
|user3  |2022-02-04|2023-02-04|inactive   |2024-02-04|1b0232de-8177-44f2-8f51-708c359f4a90|Y           |2024-02-11 13:29:12.720104|NULL                      |
|user1  |2023-02-05|2024-02-02|active     |NUL

In [32]:
bronze_users.show()

+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
|user_id|created_at|updated_at|user_status|deleted_at| id|current_flag|effective_date|expired_date|
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+



In [33]:
# check the change record 

In [34]:
# update to delta
silverDeltaTable.alias("oldData") \
    .merge(
        new_updated_data.alias("newData"),
        (col("oldData.user_id") == col("newData.user_id"))
        
    ) \
    .whenMatchedUpdate(
        condition=(
            (col("oldData.updated_at") != col("newData.updated_at"))),
            set = {
        "current_flag": "newData.current_flag",
        "expired_date": "newData.expired_date"
    }
    ) \
    .execute()

In [35]:
silverDeltaTable.toDF().show(10, False, False)

+-------+----------+----------+-----------+----------+------------------------------------+------------+--------------------------+--------------------------+
|user_id|created_at|updated_at|user_status|deleted_at|id                                  |current_flag|effective_date            |expired_date              |
+-------+----------+----------+-----------+----------+------------------------------------+------------+--------------------------+--------------------------+
|user4  |2024-02-01|2024-02-05|inactive   |2024-02-05|ec52cf68-1881-476a-b68b-96d1d85c5a65|N           |2024-02-11 13:29:16.612573|2024-02-11 13:29:16.612573|
|user5  |2023-02-05|2024-02-05|active     |NULL      |fedf0115-ff10-4f5e-bd9b-32976415cd56|Y           |2024-02-11 13:29:16.612573|NULL                      |
|user3  |2022-02-04|2023-02-04|inactive   |2024-02-04|1b0232de-8177-44f2-8f51-708c359f4a90|Y           |2024-02-11 13:29:12.720104|NULL                      |
|user1  |2023-02-05|2024-02-02|active     |NUL

In [36]:
new_updated_data.show(10,False,False)

+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
|user_id|created_at|updated_at|user_status|deleted_at|id |current_flag|effective_date|expired_date|
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+



In [37]:
silver_users.show(10,False,False)

+-------+----------+----------+-----------+----------+------------------------------------+------------+--------------------------+--------------------------+
|user_id|created_at|updated_at|user_status|deleted_at|id                                  |current_flag|effective_date            |expired_date              |
+-------+----------+----------+-----------+----------+------------------------------------+------------+--------------------------+--------------------------+
|user4  |2024-02-01|2024-02-05|inactive   |2024-02-05|ec52cf68-1881-476a-b68b-96d1d85c5a65|N           |2024-02-11 13:29:16.612573|2024-02-11 13:29:16.612573|
|user5  |2023-02-05|2024-02-05|active     |NULL      |fedf0115-ff10-4f5e-bd9b-32976415cd56|Y           |2024-02-11 13:29:16.612573|NULL                      |
|user3  |2022-02-04|2023-02-04|inactive   |2024-02-04|1b0232de-8177-44f2-8f51-708c359f4a90|Y           |2024-02-11 13:29:12.720104|NULL                      |
|user1  |2023-02-05|2024-02-02|active     |NUL

In [38]:
update_new_record.show(10,False,False)

+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
|user_id|created_at|updated_at|user_status|deleted_at|id |current_flag|effective_date|expired_date|
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+



In [39]:
new_records.show(10,False,False)

+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
|user_id|created_at|updated_at|user_status|deleted_at|id |current_flag|effective_date|expired_date|
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+



In [40]:
silverDeltaTable.alias("oldData") \
    .merge(
        update_new_record.alias("newData"),
        (col("oldData.id") == col("newData.id"))
    ) \
    .whenNotMatchedInsertAll() \
    .execute()

In [41]:
silverDeltaTable.toDF().show(20,False,False)

+-------+----------+----------+-----------+----------+------------------------------------+------------+--------------------------+--------------------------+
|user_id|created_at|updated_at|user_status|deleted_at|id                                  |current_flag|effective_date            |expired_date              |
+-------+----------+----------+-----------+----------+------------------------------------+------------+--------------------------+--------------------------+
|user4  |2024-02-01|2024-02-05|inactive   |2024-02-05|ec52cf68-1881-476a-b68b-96d1d85c5a65|N           |2024-02-11 13:29:16.612573|2024-02-11 13:29:16.612573|
|user5  |2023-02-05|2024-02-05|active     |NULL      |fedf0115-ff10-4f5e-bd9b-32976415cd56|Y           |2024-02-11 13:29:16.612573|NULL                      |
|user3  |2022-02-04|2023-02-04|inactive   |2024-02-04|1b0232de-8177-44f2-8f51-708c359f4a90|Y           |2024-02-11 13:29:12.720104|NULL                      |
|user1  |2023-02-05|2024-02-02|active     |NUL

In [42]:
silver_users.show(10,False,False)

+-------+----------+----------+-----------+----------+------------------------------------+------------+--------------------------+--------------------------+
|user_id|created_at|updated_at|user_status|deleted_at|id                                  |current_flag|effective_date            |expired_date              |
+-------+----------+----------+-----------+----------+------------------------------------+------------+--------------------------+--------------------------+
|user4  |2024-02-01|2024-02-05|inactive   |2024-02-05|ec52cf68-1881-476a-b68b-96d1d85c5a65|N           |2024-02-11 13:29:16.612573|2024-02-11 13:29:16.612573|
|user5  |2023-02-05|2024-02-05|active     |NULL      |fedf0115-ff10-4f5e-bd9b-32976415cd56|Y           |2024-02-11 13:29:16.612573|NULL                      |
|user3  |2022-02-04|2023-02-04|inactive   |2024-02-04|1b0232de-8177-44f2-8f51-708c359f4a90|Y           |2024-02-11 13:29:12.720104|NULL                      |
|user1  |2023-02-05|2024-02-02|active     |NUL

In [43]:
bronze_users.show(10,False,False)

+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
|user_id|created_at|updated_at|user_status|deleted_at|id |current_flag|effective_date|expired_date|
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+
+-------+----------+----------+-----------+----------+---+------------+--------------+------------+



In [44]:
spark.stop()